In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import os.path as osp
import yaml
import sys
import logging
from glob import glob
from torch.utils.data.dataloader import DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import average_precision_score
sys.path.append("../src")


logger = logging.getLogger(__name__)


In [ ]:
from dataset_utils import get_datasets
from lit_utils import LitModel


In [ ]:
paths = [
    "../outputs/train_model_multirun_Clothing_Shoes_and_Jewelry_20211020_172806/train_model_Clothing_Shoes_and_Jewelry_20211020_172806_0",
    "../outputs/train_model_multirun_Clothing_Shoes_and_Jewelry_20211020_172806/train_model_Clothing_Shoes_and_Jewelry_20211020_172806_1",
    "../outputs/train_model_multirun_Clothing_Shoes_and_Jewelry_20211020_172806/train_model_Clothing_Shoes_and_Jewelry_20211020_172806_2",
    "../outputs/train_model_multirun_Clothing_Shoes_and_Jewelry_20211020_172806/train_model_Clothing_Shoes_and_Jewelry_20211020_172806_3"]


In [ ]:
res_dict = {}
recall_dict, hit_dict = {}, {}
for path in paths:
    print(path)
    config_path = osp.join(path, ".hydra", "config.yaml")
    model_path = glob(osp.join(path, "epoch=*.ckpt*"))[0]

    # Load config file
    with open(config_path, "r") as f:
        cfg = yaml.safe_load(f)
    cfg["is_pretrained"] = False
    cfg["batch_size"] = 128
    print(f'{cfg["cf_weight"]=} {cfg["labeled_ratio"]=}')

    # Load dataset
    train_dataset, test_dataset, dataset_meta, pos_weight = get_datasets(
        cfg["train_df_path"],
        cfg["test_df_path"],
        cfg["cf_vector_df_path"],
        cfg["labeled_ratio"],
    )
    logger.info(
        "Sizes [trainset testset num_classes]=[{} {} {}]".format(
            dataset_meta["train_set_size"],
            dataset_meta["test_set_size"],
            dataset_meta["num_classes"],
        )
    )

    testloader = DataLoader(
        test_dataset, batch_size=cfg["batch_size"], num_workers=cfg["num_workers"]
    )

    # Load model
    lit_h = LitModel.load_from_checkpoint(model_path)
    device = "cuda:0"
    lit_h = lit_h.to(device)
    lit_h = lit_h.eval()

    # Get predictions
    label_list, pred_list = [], []
    for batch in tqdm(testloader):

        (
            imgs,
            _,
            labels,
            _,
        ) = batch

        imgs = imgs.to(device)
        y_hat, _ = lit_h(imgs)
        preds = torch.sigmoid(y_hat)

        pred_list.append(preds.detach().cpu())
        label_list.append(labels.detach().cpu())
    preds = torch.vstack(pred_list).numpy()
    labels = torch.vstack(label_list).numpy()

    # Score
    ap = average_precision_score(labels, preds, average=None)

    # Recall
    recall, hit = 0, 0
    items = 0
    no_labels = 0
    for pred, label in zip(torch.tensor(preds), torch.tensor(labels)):
        _, pred_idx = torch.topk(pred, k=5)
        label_idx = torch.where(label == 1)[0]

        if len(label_idx) == 0:
            no_labels += 1
            continue

        recall_i = sum(el in pred_idx for el in label_idx) / len(label_idx)
        recall += recall_i

        hit_i = sum(el in label_idx for el in pred_idx)
        hit += hit_i

        items += 1

    recall /= items
    hit /= items

    recall_dict[f'cf_weight={cfg["cf_weight"]} labeled_ratio={cfg["labeled_ratio"]}'] = recall
    hit_dict[f'cf_weight={cfg["cf_weight"]} labeled_ratio={cfg["labeled_ratio"]}'] = hit

    print(f"{no_labels=}")


In [ ]:
recall_dict

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(30,10))
ax = res_df[['cf_weight=0.0', 'cf_weight=2.0']].plot.bar(rot=0,ax=ax)
plt.show()

In [ ]:
pd.set_option('display.max_rows', res_df.shape[0]+1)

df = res_df[['cf_weight=0.0', 'cf_weight=2.0', 'count']]
df['diff'] =  df['cf_weight=2.0'] - df['cf_weight=0.0']
df = df.sort_values(by='diff')
df= df.reset_index()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(30,10))
ax = df[['cf_weight=0.0', 'cf_weight=2.0']].plot.bar(rot=0,ax=ax)
plt.show()

In [ ]:
plt.plot(df['count'])

In [ ]:
topk= 5

idx_top = preds.argsort()[::-1][:topk]
label_top = labels.argsort()[::-1][:topk]

In [ ]:
_, idx_top =  torch.topk(torch.tensor(preds),k=topk,dim=-1)

In [ ]:
for n,idxs in enumerate(idx_top):
    label_in_idx_top = labels[n][idxs]
    break

In [ ]:
from torchmetrics.functional import recall


In [ ]:
recall(torch.tensor(preds),torch.tensor(labels),top_k=5,average=None,num_classes=preds.shape[-1]).mean()

In [ ]:
a = [[0.99,0.99,0.99,0.99,0.99,0,0,0]]
b = [[1,1,1,1,1,0,0,0]]
recall(torch.tensor(a),torch.tensor(b),top_k=5,average=None,num_classes=8)

In [ ]:
_,idx = torch.topk(torch.tensor(a),k=5)

In [ ]:
pred_idx,label_idx

In [ ]:
recall_i